In [3]:
import re 
import sys 
import os 
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import time 
from math import log2

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[2], 'GPU')
    except RuntimeError as e:
        # Visible devices must be set at program startup
        print(e)

In [5]:
# 1. https://www.tensorflow.org/text/tutorials/text_generation
# 2. https://www.thepythoncode.com/article/text-generation-keras-python
# 3. https://www.kaggle.com/code/hommelette/neural-language-model-with-tensorflow

In [4]:
lang  = 'cwe' # 'sw'
path_file = os.path.join('data', lang+'-train.txt')
cwe = open(path_file, 'r').read().lower()
cwe_decoded =  open(path_file, 'rb').read().decode(encoding = 'utf-8')


In [5]:
unique_chars = ' !"\'(),-.0123456789:;?abcdefghijklmnopqrstuvwxyz'

In [6]:
character_list = re.findall(r"[%s]"%unique_chars, cwe_decoded)

In [8]:
chars = tf.strings.unicode_split(cwe_decoded.split(), input_encoding='UTF-8')

In [9]:
chars[0]

<tf.Tensor: shape=(7,), dtype=string, numpy=array([b'c', b'h', b'i', b'k', b'a', b'l', b'e'], dtype=object)>

In [10]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(unique_chars), mask_token=None)

In [11]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[25, 30, 31, 33, 23, 34, 27], [44, 31, 36, 37, 29, 31, 34, 27],
 [28, 23, 36, 23], ..., [36, 47, 31, 36, 29, 31], [33, 43, 28, 37, 41, 23],
 [45, 37, 41, 27, 9]]>

In [12]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


In [13]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'c', b'h', b'i', b'k', b'a', b'l', b'e'],
 [b'v', b'i', b'n', b'o', b'g', b'i', b'l', b'e'],
 [b'f', b'a', b'n', b'a'], ..., [b'n', b'y', b'i', b'n', b'g', b'i'],
 [b'k', b'u', b'f', b'o', b's', b'a'], [b'w', b'o', b's', b'e', b'.']]>

In [14]:
tf.strings.reduce_join(chars, axis=-1).numpy()


array([b'chikale', b'vinogile', b'fana', ..., b'nyingi', b'kufosa',
       b'wose.'], dtype=object)

In [15]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis = -1)

In [16]:
all_ids = ids_from_chars(tf.strings.unicode_split(cwe_decoded, 'UTF-8'))
all_ids

<tf.Tensor: shape=(603432,), dtype=int64, numpy=array([25, 30, 31, ..., 27,  9,  0])>

In [17]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)


In [18]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))


c
h
i
k
a
l
e
 
v
i


In [19]:
seq_length = 100

In [20]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'c' b'h' b'i' b'k' b'a' b'l' b'e' b' ' b'v' b'i' b'n' b'o' b'g' b'i'
 b'l' b'e' b' ' b'f' b'a' b'n' b'a' b' ' b'v' b'i' b'y' b'a' b' ' b'w'
 b'a' b'n' b'h' b'u' b' ' b'w' b'o' b'c' b'h' b'i' b'k' b'a' b'l' b'a'
 b' ' b'h' b'a' b'b' b'u' b'n' b'g' b"'" b'h' b'u' b'k' b'e' b' ' b'i'
 b'm' b'i' b's' b'i' b',' b' ' b's' b'i' b' ' b'k' b'w' b'a' b' ' b'u'
 b'm' b'e' b'l' b'o' b' ' b'n' b'a' b' ' b'u' b'n' b'g' b"'" b'w' b'a'
 b'j' b'i' b' ' b'n' b'a' b' ' b'u' b'n' b'y' b'o' b'l' b'o' b'd' b'o'
 b' ' b'n' b'a'], shape=(101,), dtype=string)


In [21]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b"chikale vinogile fana viya wanhu wochikala habung'huke imisi, si kwa umelo na ung'waji na unyolodo na"
b' uwashelati na ndwagi hebu migongo. one muhapula chinhu chochose kwa zina jangu nizamtendelani. yesu '
b'kamulongela petili, "bweleza zele jako muna iyala. "ufalume wa kuulanga ulinga vino. niye mulala mula'
b'ngulizi, nokwandikila weye gayo mulondwa wangu yonikulonda muna ikweli. nomulongelani ukweli, munhu y'
b'oyose yohauhokela ufalume wa mulungu fana mwana mdoododo, hezakwingila muna ufalume uwo ng\'o!" iyo ni'


In [22]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [23]:
split_input_target(list("Tensorflow"))


(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [24]:
dataset = sequences.map(split_input_target)


In [25]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"chikale vinogile fana viya wanhu wochikala habung'huke imisi, si kwa umelo na ung'waji na unyolodo n"
Target: b"hikale vinogile fana viya wanhu wochikala habung'huke imisi, si kwa umelo na ung'waji na unyolodo na"


In [26]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [57]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 16

# Number of RNN units
rnn_units = 128

In [58]:
class Model(tf.keras.Model):
    def __init__(self, char_size, embedding_size, run_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

In [85]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm1 = tf.keras.layers.LSTM(rnn_units, return_sequences= True, return_state= True)
                                      # return_sequences=True,
                                      # return_state=True)
#         self.lstm2 = tf.keras.layers.LSTM(rnn_units, return_sequences= True, return_state= True)
                                       #return_sequences=True)
        self.dense1 = tf.keras.layers.Dense(vocab_size//2)
        self.output_dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            h_states, c_states = self.lstm1.get_initial_state(x)
        x, h_states, c_states = self.lstm1(x, initial_state=[h_states, c_states], training=training)
#         x, h_states, c_states = self.lstm2(x, initial_state=[h_states, c_states], training=training)
        x = self.dense1(x, training=training)
        x = self.output_dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [86]:
a= tf.keras.layers.LSTM(4)(np.random.rand(2,3,3))

lstm1 = tf.keras.layers.LSTM(4)

In [87]:
lstm1(np.random.rand(2,3,3), initial_state=[a,a], training=False)

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[-0.35602093, -0.20855562,  0.177973  ,  0.0135634 ],
       [-0.12484133, -0.08875249,  0.05124971,  0.04876575]],
      dtype=float32)>

In [88]:
tf.keras.layers.LSTM(10,).get_initial_state(np.random.rand(2,3,3))

[<tf.Tensor: shape=(2, 10), dtype=float64, numpy=
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])>,
 <tf.Tensor: shape=(2, 10), dtype=float64, numpy=
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])>]

In [89]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [90]:
for input_example_batch, target_example_batch in dataset.take(1):
    print(input_example_batch.shape)
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
    

(64, 100)
(64, 100, 49) # (batch_size, sequence_length, vocab_size)


In [91]:
model.summary()

Model: "my_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     multiple                  784       
                                                                 
 lstm_15 (LSTM)              multiple                  74240     
                                                                 
 dense_6 (Dense)             multiple                  3096      
                                                                 
 dense_7 (Dense)             multiple                  1225      
                                                                 
Total params: 79,345
Trainable params: 79,345
Non-trainable params: 0
_________________________________________________________________


In [92]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 49), dtype=float32, numpy=
array([[ 5.3509808e-04, -5.1075115e-04, -7.4462773e-04, ...,
         6.7809928e-04,  4.9313563e-05,  1.6061444e-03],
       [ 1.3862356e-03, -2.3724360e-03, -1.8804010e-03, ...,
         3.0893381e-04, -9.2333124e-04,  1.4096973e-03],
       [ 4.6661354e-04, -1.2479482e-03, -5.4709567e-03, ...,
        -2.2954229e-04,  6.7879882e-04, -1.3494439e-04],
       ...,
       [-2.0664730e-03,  6.1197313e-03, -3.3908593e-03, ...,
        -4.3995762e-03,  1.2202952e-03, -5.3355759e-03],
       [-1.4811212e-03,  3.9259065e-03, -3.4192675e-03, ...,
        -2.9552651e-03,  5.8323884e-04, -2.3123610e-03],
       [-9.7479159e-04,  3.2169304e-03, -2.7935251e-03, ...,
        -3.2488327e-03, -1.4007054e-03, -1.7196579e-04]], dtype=float32)>

In [94]:
# tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [95]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [96]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'wetu yesu. kwaviya habule munhu yoyose vichili yokala kwa chimu chake mwenyewo, hebu yodanganika kwa'

Next Char Predictions:
 b'871pck9:!o!fpf!2eq13w-6[UNK]jjh;jp!zrlr !pp23,c"z5m("h?oyy?-742 pvww-gki2p" dmslr7tsj,i?e?odbv,dv?nsu!b0'


In [97]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [98]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 49)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(3.8919, shape=(), dtype=float32)


In [99]:
tf.exp(example_batch_mean_loss).numpy()

49.00391

In [100]:
model.compile(optimizer='adam', loss=loss)


In [101]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [102]:
EPOCHS = 20


In [103]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
93/93 [==============================] - 4s 12ms/step - loss: 3.1071
Epoch 2/20
93/93 [==============================] - 2s 12ms/step - loss: 2.6705
Epoch 3/20
93/93 [==============================] - 2s 12ms/step - loss: 2.2169
Epoch 4/20
93/93 [==============================] - 2s 12ms/step - loss: 2.0756
Epoch 5/20
93/93 [==============================] - 2s 13ms/step - loss: 2.0032
Epoch 6/20
93/93 [==============================] - 2s 12ms/step - loss: 1.9425
Epoch 7/20
93/93 [==============================] - 2s 12ms/step - loss: 1.8858
Epoch 8/20
93/93 [==============================] - 2s 12ms/step - loss: 1.8375
Epoch 9/20
93/93 [==============================] - 2s 12ms/step - loss: 1.7970
Epoch 10/20
93/93 [==============================] - 2s 12ms/step - loss: 1.7600
Epoch 11/20
93/93 [==============================] - 2s 12ms/step - loss: 1.7252
Epoch 12/20
93/93 [==============================] - 2s 12ms/step - loss: 1.6938
Epoch 13/20
93/93 [==================

In [105]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

  @tf.function
  def generate_one_step_probablity(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

#     Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, predicted_logits, states

In [106]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [116]:
start = time.time()
states = None
next_char = tf.constant(['r'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

r:o chisimla ma mwokakolibulelolo ya vyolobani muligo zizegakudalulonguf;ewigolivizili wawe gwa ku, nahahu delusi chi yaja yo bwi nananosa kudihemangi! tulo, wa wa mwazabusnda "-jetinanfr6-nalulegunguhinolale ji wilina wiga c'u awi, cha za wahu mudag'oza ka hikanala. ka wili fa, :qga umi sbile yo mwaka mulo vindole, c'habawizase! ", ma bugale yiche m99hekalizona, '9liku dinvilo. 'o mgosziluli ya de, kulo a tivinisumwaa mavi mwa kulalamuse wo davetadilule a yalegwamaba mhe? weku uwabu.", kwilikam'3gho ba weli! mgigit'vibbe. ze, komwa ya za mjwelesu f(pake lo-" mutalisi wabulanajaka mf? wawako, ka dilo" dending'welaha f7konilo va, nhima mko yajakulumke we hizu mumwa mu, ku nale, mu, gu vegi u nogutena hulwa msilakwangi ka. kwa ho mpa kammu mwalalo wililawa ndilesindi daki ila nadalanagu, svi waluwo, mdizenikulahilo ya ha yahulute, ta wagaulilalelwa memhigango? yama wa bachi mwe4ja wele na vananokale ze, chwanamandizi mesamundi, vanombule kahi zanha mwo mawigu co welo choma wa vi vaba nf;

In [108]:
loss = 0 

start = time.time()
states = None
next_char = tf.constant(['r'])
result = []

total_loss_list = []
for n in range(100):
    next_char, next_probablity, states = one_step_model.generate_one_step_probablity(next_char, states=states)
    total_loss_list.append(-np.log2(max(tf.nn.softmax(next_probablity.numpy()[0], axis=None))))
    result.append(next_probablity)

# result = tf.strings.join(result)
end = time.time()
# print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
# print('\nRun time:', end - start)



In [109]:
np.mean(total_loss_list)

1.8515096

In [110]:
# loss_from_scratch -= log2(from_scratch_model(lang, c, history))

max(tf.nn.softmax(
    result[3][0].numpy(), axis=None, name=None
))

<tf.Tensor: shape=(), dtype=float32, numpy=0.4233668>

In [111]:
max(tf.nn.softmax(
    result[3].numpy(), axis=None, name=None
))

<tf.Tensor: shape=(49,), dtype=float32, numpy=
array([0.00000000e+00, 4.23366815e-01, 3.88109894e-03, 2.11546780e-03,
       2.31456695e-04, 1.61363914e-05, 1.03647391e-04, 2.11393703e-02,
       6.17158716e-04, 1.25449784e-02, 3.89075140e-04, 1.18752796e-04,
       1.78890157e-04, 8.66917035e-05, 2.94219179e-04, 1.99685397e-04,
       1.02464779e-04, 5.67981479e-05, 3.37259436e-04, 1.43309342e-04,
       5.56725936e-05, 4.62811440e-05, 4.51324275e-03, 2.65933550e-03,
       4.14857231e-02, 3.26167280e-03, 6.86343247e-03, 6.77686912e-05,
       1.03892675e-02, 1.94068681e-02, 2.33222339e-02, 4.65103425e-03,
       6.45865174e-03, 5.63498102e-02, 9.11672115e-02, 3.99466045e-02,
       1.08745918e-01, 9.97045310e-04, 4.04702825e-03, 9.59546596e-05,
       6.35957476e-05, 2.85527781e-02, 1.16171837e-02, 8.63710884e-03,
       1.48010449e-02, 2.69457325e-02, 1.58236595e-04, 6.60815835e-03,
       1.21620782e-02], dtype=float32)>

In [290]:
def evaluate_one(lang):
    testfile = open(os.path.join('data',lang+'-test.txt'), 'r')
    max_history = 100
#     history = []
    loss_anything_goes = 0
    loss_from_scratch = 0
    count = 0
    states = None
    total_loss_list = []
    while count<1000:
        c = testfile.read(1)
        if not c:
            break
        
        if count ==0:
            next_char = tf.strings.unicode_split(c, input_encoding='UTF-8')
        else: 
            next_char = tf.strings.unicode_split(c, input_encoding='UTF-8')
        count += 1
        next_char, next_probablity, states = one_step_model.generate_one_step_probablity(next_char, states=states)
        total_loss_list.append(-np.log2(np.max(tf.nn.softmax(next_probablity.numpy()[0]))))
#         if len(history) == max_history:
#             history.pop(0)
#         history.append(c)
    return total_loss_list
        
#         loss_anything_goes -= log2(anything_goes_model(lang, c, history))
#         loss_from_scratch -= log2(from_scratch_model(lang, c, history))
#         if len(history) == max_history:
#             history.pop(0)
#         history.append(c)
#     return [loss_from_scratch/count, loss_anything_goes/count]
     



In [293]:
next_probablity.numpy()[0]

array([       -inf,  4.7544885 ,  0.06236807, -0.5444741 , -2.7571123 ,
       -5.420428  , -3.5605106 ,  1.7573867 , -1.7763793 ,  1.2355703 ,
       -2.237733  , -3.4244611 , -3.0147336 , -3.739147  , -2.5171802 ,
       -2.9047627 , -3.5719862 , -4.162002  , -2.3806531 , -3.2364998 ,
       -4.182018  , -4.3667707 ,  0.21326596, -0.31567425,  2.4315991 ,
       -0.11150975,  0.6324579 , -3.9854047 ,  1.0470232 ,  1.6718768 ,
        1.8556572 ,  0.24333946,  0.5716704 ,  2.7378285 ,  3.2189455 ,
        2.3937936 ,  3.3952641 , -1.2967088 ,  0.10423307, -3.6376297 ,
       -4.048959  ,  2.058004  ,  1.1587355 ,  0.8623179 ,  1.4009476 ,
        2.0000749 , -3.137414  ,  0.5945552 ,  1.2045727 ], dtype=float32)

In [294]:
loss = evaluate_one('cwe')

In [295]:
sum(loss)/len(loss)

1.8064546411186457